In [1]:
import numpy as np
import pandas as pd

In [2]:
file = "C:/Users/María Moya/Documents/GitHub/PROJECT_EDA_JOAQUINVILLAR_ONLINE_DS_THEBRIDGE/src/data/Hello_Auto_data.xlsx"

In [3]:
df_vehicles = pd.read_excel(file,sheet_name = "Vehicles")
df_vehicles
df_vehicles.sample(12)

,policyId,vehicleId,brand,fuel,power,value,isActive,vehicleAge,useDescription,garageTypeDescription,vehicletype
47345,23730,70238,CITROEN,Diésel,163,34886.0,1,14.0,Particular Ocasional,Vía Pública,Turismo
59296,30781,84517,MAZDA,Diésel,109,20765.0,0,19.0,Particular Ocasional,Comunitario,Turismo
15061,7001,31251,RENAULT,Diésel,92,15965.0,0,21.0,Privado,Vía Pública,Turismo
51845,26043,75260,CITROEN,Diésel,90,23678.0,1,20.0,Particular Habitual (diario),Vía Pública,Turismo
27375,13413,47044,TOYOTA,Híbrido,100,20300.0,0,5.0,Particular Habitual (diario),Comunitario,Turismo
24087,11595,42988,SEAT,Diésel,90,11529.0,1,18.0,Particular Habitual (diario),Vía Pública,NaN
55409,28171,79580,RENAULT,Diésel,65,13552.0,1,17.0,Particular Ocasional,Vía Pública,Turismo
80530,53824,257723,BMW,Diésel,110,22588.0,1,NaN,Particular Ocasional,Vía Pública,NaN
39782,19864,66267,FORD,Gasolina,208,34111.0,1,6.0,Particular Habitual (diario),Individual,Turismo
20624,9795,38728,FORD,Diésel,90,18970.0,1,13.0,Privado,Individual,Turismo


In [4]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80972 entries, 0 to 80971
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   policyId               80972 non-null  int64  
 1   vehicleId              80972 non-null  int64  
 2   brand                  80972 non-null  object 
 3   fuel                   80972 non-null  object 
 4   power                  80972 non-null  int64  
 5   value                  77055 non-null  float64
 6   isActive               80972 non-null  int64  
 7   vehicleAge             79629 non-null  float64
 8   useDescription         79128 non-null  object 
 9   garageTypeDescription  80972 non-null  object 
 10  vehicletype            75712 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 6.8+ MB


In [5]:
df_vehicles.fuel.value_counts()

fuel
Diésel         51321
Gasolina       27341
Híbrido         1012
Otros           1009
Eléctrico        255
X                 11
Not Allowed       11
GLP                5
P                  4
L                  3
Name: count, dtype: int64

# Análisis Univariante
1.1. Distribución del combustible de los vehículos \
1.2. Distribución del año de los vehículos \
1.3. Distribución del tipo de vehículo